# An introduction to remote calls in Julia

First add some workers

In [1]:
if nprocs() == 1 
    addprocs(2)
end;
workers()

2-element Array{Int64,1}:
 2
 3

Now we create an anonymous function `() -> "Allez les Bleus"` and ask Julia to execute it on worker `2` with `remoteall`.

In [5]:
r = remotecall(() -> "Allez les Bleus", 2)

Future(2, 1, 7, Nullable{Any}())

The result return from `remotecall` is a `Future`. The `Future` keeps a reference to the the actual result of the computation. To result can be retrieved with the `fetch` function.

In [6]:
fetch(r)

"Allez les Bleus"

but this doesn't have to happen on the master node

In [7]:
remotecall(() -> println(fetch(r)), 2);

	From worker 2:	Allez les Bleus


In some cases, this could also happen from another worker process but JuliaBox is by default set up to only allow master-worker communication so

In [8]:
remotecall(() -> println(fetch(r)), 3);

	From worker 3:	Allez les Bleus


will fail on JuliaBox but not if you e.g. `addprocs(2)` locally.

In [26]:
r = remotecall(() -> myid(), 3)
@show fetch(r)
remotecall(() -> println(fetch(r)), 2);

fetch(r) = 3
	From worker 2:	3


In [28]:
r = remotecall(i -> i + myid(), 2, 1)
fetch(r)

3

## Tasks

The `remotecall` is asynchronous and synchronization will happen when the result is fetched. Often, it is useful to be able to control the synchronization more precisely. This is possible with the two related functions `remotecall_fetch` and `remotecall_wait`. Their behaviors are very similar to `remotecall` but both are blocking. However, they may be combined with `@async` and `@sync` annotations to setup blocks of code to be run asynchronously and when sync up again. The difference between the two blocking versions is that `remotecall_fetch` will return the result whereas `remotecall_wait` will return a `Future` with a reference to the result.

The blocking behavior can be tested with the `sleep` function

In [31]:
@time remotecall(() -> sleep(2), 2);

  0.000190 seconds (91 allocations: 4.172 KiB)


In [32]:
@time fetch(remotecall(() -> sleep(2), 2));

  2.041510 seconds (228 allocations: 7.609 KiB)


In [34]:
@time typeof(remotecall_wait(() -> sleep(2), 2))

  2.098860 seconds (286 allocations: 12.500 KiB)


Future

In [35]:
@time typeof(remotecall_fetch(() -> sleep(2), 2))

  2.010359 seconds (1.36 k allocations: 72.019 KiB)


Void

By annotating remotecall_fetch with `@async`, we can make the code block return immediately

In [40]:
@time @async remotecall_fetch(() -> (sleep(2); 1), 2)

  0.000080 seconds (36 allocations: 2.950 KiB)


Task (runnable) @0x000000011298e1d0

This can be useful when we want to run concurrent computations on the available workers. First we show an example without asynchronous execution and the one with asynchronous execution

In [42]:
@time for p in workers()
    remotecall_fetch(() -> sleep(1), p)
end

  4.210647 seconds (512 allocations: 18.891 KiB)


In [43]:
@time @sync for p in workers()
    @async remotecall_fetch(() -> sleep(1), p)
end

  1.021786 seconds (4.18 k allocations: 242.341 KiB)


This is a very common pattern so Julia has a convenience function defined for exactly this

In [50]:
@time @sync asyncmap(p -> remotecall_fetch(() -> (sleep(1); myid()), p), workers())

  1.051919 seconds (17.16 k allocations: 966.828 KiB)


4-element Array{Int64,1}:
 2
 3
 4
 5

The nested anonymous functions become hard to read. This is true in particular when the function body of the anonymous functions become non-trivial. It is therefore often better to use Julia's `do` syntax for anonymous functions. An equivalent formulation of the previous call with `do` syntax would look like

In [51]:
@time @sync asyncmap(workers()) do p
    remotecall_fetch(p) do
        return (sleep(1); myid())
    end
end

  1.051252 seconds (17.05 k allocations: 963.638 KiB)


4-element Array{Int64,1}:
 2
 3
 4
 5

It might take a little effort to get used to the `do` syntax but doing so

In [17]:
[remotecall_fetch(() -> myid(), i) for i in workers()]

4-element Array{Int64,1}:
 2
 3
 4
 5

In [18]:
remotecall_wait(3) do
    println("do syntax")
end

	From worker 3:	do syntax


Future(3, 1, 24, Nullable{Any}())